A notebook to produce charge windows from train station-ending journey stages from UK NTS data
24 Feb - J Dixon

In [24]:
import pandas as pd

In [25]:
# import TRIP data from stata file
def return_trip_data_from_nts(years=[2022]):
    df = pd.read_stata(f'./nts_data/UKDA-5340-stata/stata/stata13/trip_eul_2002-2022.dta')
    df = df[(df['SurveyYear'].isin(years))]
    df = df[(df['TripStart'] != 'NA') & (df['TripEnd'] != 'NA')]
    return df

trips = return_trip_data_from_nts()
trips.head()

ERROR! Session/line number was not unique in database. History logging moved to new session 33


,TripID,DayID,IndividualID,HouseholdID,PSUID,PersNo,TravDay,JourSeq,HowComp_B01ID,SeriesCall_B01ID,...,TripTravTime_B01ID,TripOrigGOR_B02ID,TripDestGOR_B02ID,JJXSC,JOTXSC,JTTXSC,JD,W5,W5xHH,SurveyYear
5043955,2022000001,2022000001,2022000001,2022000001,2022000002,1,1,1,From diary,No,...,15 minutes to under 30 minutes,South East,8,1,15,15,6.0,0.61702412455686,1,2022
5043956,2022000002,2022000001,2022000001,2022000001,2022000002,1,1,2,From diary,No,...,15 minutes to under 30 minutes,South East,8,1,15,15,6.0,0.61702412455686,1,2022
5043957,2022000003,2022000001,2022000001,2022000001,2022000002,1,1,3,From diary,No,...,3 minutes to under 8 minutes,South East,8,1,5,5,2.0,0.61702412455686,1,2022
5043958,2022000004,2022000001,2022000001,2022000001,2022000002,1,1,4,From diary,No,...,3 minutes to under 8 minutes,South East,8,1,5,5,2.0,0.61702412455686,1,2022
5043959,2022000005,2022000002,2022000001,2022000001,2022000002,1,2,1,From diary,No,...,3 minutes to under 8 minutes,South East,8,1,5,5,1.0,0.67632383372503,1.09610598161094,2022


In [26]:
# import STAGE data from stata file. Specify years
def return_stage_data_from_nts(years=[2022]):
    df = pd.read_stata(f'./nts_data/UKDA-5340-stata/stata/stata13/stage_eul_2002-2022.dta')
    df = df[(df['SurveyYear'].isin(years))]
    df = df[(df['StageTime'] != 'NA')]
    return df

stages = return_stage_data_from_nts()
stages.head()

,StageID,TripID,DayID,IndividualID,HouseholdID,PSUID,VehicleID,IndTicketID,PersNo,TravDay,...,StagePassCost,StagePassCost_B01ID,StageCost,StageCost_B01ID,StageMain_B01ID,SSXSC,STTXSC,SD,WhereParked_B01ID,SurveyYear
5278371,2022006517,2022006292,2022003394,2022000562,2022000248,2022000043,2022000319,NA,1,6,...,NA,DNA,NA,DNA,Yes,1,5,0.5,DEAD,2022
5278372,2022006518,2022006293,2022003394,2022000562,2022000248,2022000043,2022000319,NA,1,6,...,NA,DNA,NA,DNA,Yes,1,15,9.0,DEAD,2022
5278373,2022006535,2022006310,2022003403,2022000565,2022000250,2022000043,2022000322,NA,1,1,...,NA,DNA,NA,DNA,Yes,1,10,4.5,DEAD,2022
5278374,2022006536,2022006311,2022003403,2022000565,2022000250,2022000043,2022000322,NA,1,1,...,NA,DNA,NA,DNA,Yes,1,10,4.5,DEAD,2022
5278375,2022006537,2022006312,2022003403,2022000565,2022000250,2022000043,NA,NA,1,1,...,NA,DNA,NA,DNA,Yes,1,10,1.0,DEAD,2022


In [44]:
# now, the strategy is:
    # find stages where the main mode is rail
    # IF the stage before it is completed by car:
        # set the start timeF of charge event to the stage start time
        # set the energy requirement of the charge event from that car journey
        # find a return trip from that same individual ID (back in trip data)
        # set the end time of charge event to the end time of the train stage in that trip

# create a new dataframe for charge_windows
charge_windows = pd.DataFrame()

# set consumption (kWh per mile)
consumption = 0.32

# bug - some of the stage TripIDs are not in the Trips dataset
stages = stages[stages['TripID'].isin(trips['TripID'])]

# return rail stages
rail_stages = stages[stages['StageMode_B04ID'] == 'Surface Rail']  # 2022 data - 2328 rail stages

# set a counter for charge_windows
row_count = 0

for i in list(rail_stages.index): 
    
    # return stage sequence
    StageSeq = rail_stages['StageSeq'][i]
    
    # return trip ID of stage
    TripID = rail_stages['TripID'][i]  
    
    # check if any stage before this is via car
    if not stages[(stages['TripID'] == TripID) & (stages['StageSeq'] < StageSeq) & (stages['StageMode_B04ID'] == 'Car / van driver')].empty:

        # return trip from trips data
        Trip = trips[trips['TripID'] == TripID]
        
        # find start time of stage - add trip start time plus stage times
        TripStartTime = int(Trip['TripStart'].tolist()[0])
        
        # return combined time of all stages up to and including this one
        # TODO: set this as charge start time
        StageStartTime = TripStartTime + int(stages[(stages['TripID'] == TripID) & (stages['StageSeq'] <= StageSeq)]['StageTime'].sum())
        
        # TODO: set this as energy requirement
        EnergyRequirement = stages[(stages['TripID'] == TripID) & (stages['StageSeq'] < StageSeq) & (stages['StageMode_B04ID'] == 'Car / van driver')]['StageDistance'].sum() * consumption
 
        # return individualID from data
        IndividualID = rail_stages['IndividualID'][i]
        
        # return trip purpose FROM (origin)
        TripPurposeFrom = Trip['TripPurpFrom_B01ID'].tolist()[0]
        
        # return trip purpose TO (destination)
        TripPurposeTo = Trip['TripPurpTo_B01ID'].tolist()[0]
        
        # find a return trip from that same individual with the opposite set of purposes
        if not trips[(trips['IndividualID'] == IndividualID) & (trips['TripPurpFrom_B01ID'] == TripPurposeTo) & (trips['TripPurpTo_B01ID'] == TripPurposeFrom)].empty:
            ReturnTrip = trips[(trips['IndividualID'] == IndividualID) & (trips['TripPurpFrom_B01ID'] == TripPurposeTo) & (trips['TripPurpTo_B01ID'] == TripPurposeFrom)]  # may return multiple trips, so we will take the first one as the return
            
            # return ID of Return Trip
            ReturnTripID = ReturnTrip['TripID'].tolist()[0]
            
            # return start time of return trip
            ReturnTripStartTime = int(ReturnTrip['TripStart'].tolist()[0])
            
            # return stage data for return trip
            ReturnStages = stages[stages['TripID'] == ReturnTripID]
            
            # find rail stage
            if not ReturnStages[ReturnStages['StageMode_B04ID'] == 'Surface Rail'].empty:
                
                ReturnStageSeq = ReturnStages[ReturnStages['StageMode_B04ID'] == 'Surface Rail']['StageSeq'].tolist()[0]
                
                # return end time of stage
                # TODO: set this as charge event end time
                ReturnStageEndTime = ReturnTripStartTime + int(stages[(stages['TripID'] == ReturnTripID) & (stages['StageSeq'] <= ReturnStageSeq)]['StageTime'].sum())
                
                # if we get this far, we have all the data. make a charge window.
                charge_windows.at[row_count, 'StartTime_Hrs'] = StageStartTime/60
                charge_windows.at[row_count, 'EndTime_Hrs'] = ReturnStageEndTime/60
                charge_windows.at[row_count, 'EnergyRequired_kWh'] = EnergyRequirement
                
                row_count += 1

In [47]:
charge_windows

,StartTime_Hrs,EndTime_Hrs,EnergyRequired_kWh
0,11.333333,17.833333,0.48
1,10.916667,16.416667,0.96
2,19.083333,23.050000,1.60
3,9.500000,20.750000,1.92
4,5.616667,15.583333,5.12
...,...,...,...
72,8.583333,14.750000,0.32
73,8.583333,14.750000,0.32
74,8.583333,14.750000,0.32
75,13.250000,17.750000,0.48
